<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/08_document_classification_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNを使った文書分類
* RNNの出力を文書の潜在表現として利用し、文書分類を行う。
 * トークンの埋め込みも含めて学習する。

## torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 準備
* 再現性の確保については下記を参照。
 * https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import datasets

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 訓練データから語彙集合を作成
* データの読み込みや前処理は`torchtext`の旧バージョン（0.9より前）とかなり変わっているので注意。

In [ ]:
import torchtext
torchtext.__version__

'0.11.0'

* データセットの訓練データ部分をもとに語彙セットを作成する。
 * 語彙セットを作成するとき、テストデータ部分を使わないように。

In [ ]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
#tokenizer = get_tokenizer('spacy') # やや時間がかかる。

# IMDbデータの訓練データ部分のイテレータを取得
train_iter = IMDB(split='train')

# 各文書をtokenizeするヘルパ関数
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

# 最小の出現頻度をmin_freqで指定して語彙サイズを調整する
vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  specials=["<unk>", "<pad>"],
                                  min_freq=5)

# OoVトークンには"<unk>"のインデックスを返すことにする
vocab.set_default_index(vocab["<unk>"]) 

100%|██████████| 84.1M/84.1M [00:02<00:00, 38.0MB/s]


In [ ]:
print(len(vocab))

30123


In [ ]:
vocab(['here', 'is', 'an', 'example'])

[132, 10, 41, 465]

In [ ]:
vocab(['<unk>', '<pad>'])

[0, 1]

### IMDBデータを分割してデータセットを作成

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# 訓練データ部分から検証データを切り出す
num_train = int(len(train_dataset) * 0.9)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
next(iter(train_dataloader))

[('pos',
  'neg',
  'neg',
  'neg',
  'pos',
  'pos',
  'neg',
  'neg',
  'neg',
  'neg',
  'pos',
  'pos',
  'neg',
  'neg',
  'neg',
  'pos',
  'pos',
  'neg',
  'pos',
  'pos',
  'pos',
  'neg',
  'neg',
  'neg',
  'neg',
  'pos',
  'pos',
  'neg',
  'neg',
  'neg',
  'pos',
  'neg',
  'pos',
  'pos',
  'neg',
  'neg',
  'neg',
  'neg',
  'neg',
  'neg',
  'neg',
  'neg',
  'pos',
  'pos',
  'pos',
  'pos',
  'neg',
  'pos',
  'neg',
  'neg',
  'pos',
  'neg',
  'pos',
  'neg',
  'neg',
  'pos',
  'neg',
  'neg',
  'pos',
  'pos',
  'neg',
  'pos',
  'neg',
  'pos'),
 ('This 1998 film was based on a script by the late Edward D. Wood, a script that featured NO dialogue in the tradition of films such as THE THIEF. While much of Wood\'s work was quirky low-budget entries into various genre-film traditions, his first released feature GLEN OR GLENDA was a truly visionary attempt to express the inexpressible through primitive avant-garde techniques. I WOKE UP EARLY THE DAY I DIED represen

### RNNへの入力として使えるミニバッチを作る関数

In [ ]:
vocab.get_stoi()['<pad>']

1

In [ ]:
PAD_IDX = vocab.get_stoi()['<pad>']

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
  labels = {'neg':0, 'pos':1}
  label_list, text_list = [], []
  for _label, _text in batch:
    label_list.append(labels[_label])
    processed_text = torch.tensor(vocab(tokenizer(_text)), dtype=torch.int64)
    text_list.append(processed_text)
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = pad_sequence(text_list, batch_first=False, padding_value=PAD_IDX)
  return label_list.to(device), text_list.to(device)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=collate_batch)

In [ ]:
next(iter(train_dataloader))

(tensor([1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
         0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
         1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([[  13,    2,   54,  ...,    6,   13,   13],
         [ 193,  855,    2,  ..., 6369,  149,  425],
         [   8, 8788,   21,  ..., 2757,  119,    9],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0'))

In [ ]:
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)

## モデルの定義
* LSTMを使う（GRUに変えても良い）
 * http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
INPUT_DIM = len(vocab)
NUM_CLASS = 2
EMBED_DIM = 64
HIDDEN_DIM = 64

In [ ]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx, p=0.0):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.dropout = p

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(emb_dim, hid_dim)
    self.fc = nn.Linear(hid_dim * 2, num_class)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # srcの形は[単語列長, バッチサイズ]

    embedded = self.dropout(self.embedding(src))
    # embeddedの形は[単語列長, バッチサイズ, 埋め込み次元数]

    outputs, (hidden, _) = self.rnn(embedded)
    # outputsの形は[単語列長, バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[1, バッチサイズ, 隠れ状態の次元数]

    mean_outputs = outputs.mean(0)
    hidden = hidden.squeeze()
    # mean_outputsの形は[バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[バッチサイズ, 隠れ状態の次元数]
    output = self.fc(torch.cat((mean_outputs, hidden), dim=1))

    return output

* モデルのインスタンスを得る

In [ ]:
model = RNNTextSentiment(EMBED_DIM, HIDDEN_DIM, NUM_CLASS, INPUT_DIM,
                         padding_idx=PAD_IDX, p=0.5).to(device)

### 最適化アルゴリズムの設定

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

パラメータの数を数えてみる。

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,961,410 trainable parameters


### 文書分類の損失関数の設定

In [ ]:
criterion = nn.CrossEntropyLoss()

### 訓練用の関数

In [ ]:
def train(model, loader, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0.
  epoch_acc = 0.
  for label, text in loader:
    output = model(text)
    loss = criterion(output, label)
    loss.backward()

    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item()
    epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

### 評価用の関数

In [ ]:
def evaluate(model, loader, criterion):
  model.eval()

  epoch_loss = 0.
  epoch_acc = 0.
  with torch.no_grad():
    for label, text in loader:
      output = model(text)
      loss = criterion(output, label)
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time // 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## 学習の実行

In [ ]:
N_EPOCHS = 10
CLIP = 1.

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, CLIP)
  valid_loss, valid_acc = evaluate(model, valid_dataloader, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc * 100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc * 100:.1f}% (valid)')

Epoch 1 | time in 0 minutes, 49 seconds
	Loss 0.6714 (train)	|	Acc 58.0% (train)
	Loss 0.6627 (valid)	|	Acc 65.2% (valid)
Epoch 2 | time in 0 minutes, 48 seconds
	Loss 0.5949 (train)	|	Acc 69.3% (train)
	Loss 0.5840 (valid)	|	Acc 73.4% (valid)


KeyboardInterrupt: ignored

# 課題
* 上のコードを動かして、感情分析を実践してみよう。
* 余裕があれば、ハイパーパラメータをチューニングして、分類性能を上げてみよう。

## テストデータで評価

In [ ]:
loss, acc = evaluate(model, test_dataloader, criterion)
print(f'\tLoss {loss:.4f} (test)\t|\tAcc {acc * 100:.1f}% (test)')